# Agentic RAG with LangGraph (timeframe drift fix)

This notebook adds explicit control flow to fix timeframe drift: **rewrite → retrieve → grade → bounded retry → generate with citations + confidence**.


In [ ]:
from __future__ import annotations

import json
import os
from pathlib import Path

from dotenv import load_dotenv

from src.config import settings
from src.graph import build_agentic_rag_graph, run_agentic_rag
from src.rag_baseline import baseline_rag_answer
from src.retrieval import load_persisted_index

load_dotenv()
if not os.getenv('OPENAI_API_KEY'):
    raise EnvironmentError('OPENAI_API_KEY is missing. Set it in your environment or .env file.')

print('Config loaded.')
print(f"OPENAI_MODEL={settings.openai_model}")
print(f"TOP_K={settings.top_k}, MAX_RETRIES={settings.max_retries}, RECENCY_DAYS={settings.recency_days}")


In [ ]:
index_dir = Path(settings.chroma_dir)
if not index_dir.exists() or not any(index_dir.iterdir()):
    raise FileNotFoundError(
        f'Persisted index not found at {index_dir}. Run notebooks/02_indexing_chroma_llamaindex.ipynb first.'
    )

index = load_persisted_index(chroma_dir=index_dir, embed_model=settings.embed_model)
print(f'Index loaded from {index_dir}')


In [ ]:
DRIFT_QUERY = 'What embedding model should we use?'

# Optional reminder of baseline behavior from Notebook 03
baseline = baseline_rag_answer(
    index=index,
    query=DRIFT_QUERY,
    top_k=int(settings.top_k),
    model=settings.openai_model,
    temperature=float(settings.temperature),
    max_context_chars=int(settings.max_context_chars),
)
print('Baseline answer (Notebook 03 style):')
print(baseline['answer'])
print('Baseline citations:')
for c in baseline['citations']:
    print('-', c)


In [ ]:
graph = build_agentic_rag_graph(
    index=index,
    openai_model=settings.openai_model,
    temperature=float(settings.temperature),
    top_k=int(settings.top_k),
    max_context_chars=int(settings.max_context_chars),
    max_retries=int(settings.max_retries),
    recency_days=int(settings.recency_days),
    evidence_min_recent_chunks=int(settings.evidence_min_recent_chunks),
    use_llm_grader=settings.use_llm_grader == '1',
    raw_notes_dir=settings.raw_notes_dir,
)

result = run_agentic_rag(graph, DRIFT_QUERY)

print('Decision trace:')
for step in result['decision_trace']:
    print('-', step)

print('\nRewritten query:')
print(result['rewritten_query'])

print('\nRetrieved chunks (score | doc_date | doc_title | chunk_id):')
for chunk in result['retrieved_chunks']:
    print(
        f"- {chunk['score']} | {chunk['doc_date']} | {chunk['doc_title']} | {chunk['chunk_id']}"
    )

print('\nGrade + retries:')
print(f"evidence_ok={result['evidence_ok']}, retry_count={result['retry_count']}, confidence={result['confidence']}")

print('\nFinal answer payload:')
print(json.dumps(result['final_answer'], indent=2))


## What changed vs baseline?

The baseline pipeline in Notebook 03 is retrieve → generate with no explicit evidence checks, so it can drift to stale recommendations.

Notebook 04 adds explicit control flow with LangGraph:
1. rewrite query for recency intent
2. retrieve chunks
3. grade evidence for recency + relevance
4. bounded retry (`MAX_RETRIES`) if evidence is weak
5. generate grounded answer with citations + confidence (+ optional clarifying next step when confidence is low).
